In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

In [26]:
column_names = data.columns.tolist()
print(column_names)

['srch_id', 'date_time', 'site_id', 'visitor_location_country_id', 'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id', 'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool', 'prop_location_score1', 'prop_location_score2', 'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag', 'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window', 'srch_adults_count', 'srch_children_count', 'srch_room_count', 'srch_saturday_night_bool', 'srch_query_affinity_score', 'orig_destination_distance', 'random_bool', 'comp1_rate', 'comp1_inv', 'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv', 'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv', 'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv', 'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv', 'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv', 'comp6_rate_percent_diff', 'comp7_rate', 'comp7_inv', 'comp7_rate_percent_diff', 'comp8_rate', 'comp8_inv', 'comp8_rate_percent_diff', 'click_b

In [ ]:
# df = data[data['random_bool'] == 1]
#
# search_info = ['srch_id', 'date_time']
# feature_product = ['position', 'prop_starrating',
#                    'prop_location_score1', 'prop_log_historical_price', 'prop_brand_bool',
#                    'promotion_flag', 'price_usd', 'random_bool', 'booking_bool']
#
# feature_consumer = ['srch_booking_window', 'srch_length_of_stay', 'srch_adults_count',
#                     'srch_children_count', 'srch_room_count', 'srch_saturday_night_bool']
#
# feature = feature_product + feature_consumer
# df = df[search_info + feature]
# df['date_time'] = pd.to_datetime(df['date_time'])
#
# # Count occurrences of each srch_id
# srch_id_counts = df["srch_id"].value_counts()
#
# # Print the count of each srch_id
# print(srch_id_counts)
#
# # Filter and display only srch_id values that appear more than once
# duplicates = srch_id_counts[srch_id_counts >= 5]
# print("\n", duplicates)  # Show only srch_id values that occur multiple times
#
# # Keep only rows where srch_id appears more than once
# filtered_data = df[df["srch_id"].isin(duplicates.index)]
#
# # Display the filtered data
# print(filtered_data.head())

In [ ]:
# # Compute group size within each srch_id
# filtered_data["group_size"] = filtered_data.groupby("srch_id")["srch_id"].transform("count")
#
# # Compute the total number of bookings and clicks per srch_id
# group_booking_counts = filtered_data.groupby("srch_id")["booking_bool"].sum()  # Count of bookings
# group_total_counts = filtered_data.groupby("srch_id")["booking_bool"].count()  # Count of all interactions
#
# # Compute dynamic weights based on booking frequency
# booking_weight_map = (group_total_counts / group_booking_counts).fillna(1).to_dict()  # If no bookings, set weight = 1
#
# # Map weights back to the dataframe
# filtered_data["booking_weight"] = filtered_data["srch_id"].map(booking_weight_map)
# filtered_data["click_weight"] = 1  # Click weight remains 1
#
# # Use vectorized operation instead of apply()
# filtered_data["weight"] = filtered_data["booking_weight"].where(filtered_data["booking_bool"] == 1, filtered_data["click_weight"])
#
# # Compute base probability (1 / group_size)
# filtered_data["base_prob"] = 1 / filtered_data["group_size"]
#
# # Compute weighted probability
# filtered_data["weighted_prob"] = filtered_data["base_prob"] * filtered_data["weight"]
#
# # Normalize probabilities within each srch_id
# filtered_data["orig_prob"] = filtered_data.groupby("srch_id")["weighted_prob"].transform(lambda x: x / x.sum())
#
# # Drop intermediate columns if not needed
# filtered_data = filtered_data.drop(columns=["group_size", "booking_weight", "click_weight", "base_prob", "weight", "weighted_prob"])


In [31]:
# Convert DataFrame to PyTorch tensors
data_tensor = torch.tensor(data.drop(columns=["date_time"]).values, dtype=torch.float32)
random_bool_tensor = torch.tensor(data["random_bool"].values, dtype=torch.bool)
srch_id_tensor = torch.tensor(data["srch_id"].values, dtype=torch.long)
booking_bool_tensor = torch.tensor(data["booking_bool"].values, dtype=torch.float32)

# Move tensors to GPU (if available)
device = "cuda" if torch.cuda.is_available() else "cpu"

data_tensor = data_tensor.to(device)
random_bool_tensor = random_bool_tensor.to(device)
srch_id_tensor = srch_id_tensor.to(device)
booking_bool_tensor = booking_bool_tensor.to(device)

In [32]:
# Step 1: Filter where random_bool == 1
mask = random_bool_tensor
filtered_tensor = data_tensor[mask]
filtered_srch_id = srch_id_tensor[mask]
filtered_booking_bool = booking_bool_tensor[mask]

# Step 2: Compute srch_id counts (like value_counts in Pandas)
unique_ids, inverse_indices, srch_id_counts = torch.unique(filtered_srch_id, return_inverse=True, return_counts=True)

# Step 3: Filter srch_id that appear at least 5 times
valid_mask = srch_id_counts >= 5
valid_srch_ids = unique_ids[valid_mask]

In [34]:
# Step 4: Remap `srch_id_tensor` to zero-based indices
srch_id_map = {old_id: new_idx for new_idx, old_id in enumerate(unique_ids.tolist())}
mapped_srch_ids = torch.tensor([srch_id_map[i] for i in filtered_srch_id.tolist()], dtype=torch.long, device=device)

# Step 5: Compute total bookings per srch_id
booking_sums = torch.zeros(len(unique_ids), dtype=torch.float32, device=device)
booking_sums.scatter_add_(0, mapped_srch_ids, filtered_booking_bool)

# Step 6: Compute total interactions per srch_id (Fix: Ensure GPU compatibility)
total_interactions = srch_id_counts.to(device).float()

# Step 7: Compute booking weight: total interactions / total bookings
booking_weight_tensor = torch.where(
    booking_sums > 0,
    total_interactions / booking_sums,
    torch.tensor(1.0, dtype=torch.float32, device=device)  # Ensure dtype and device match
)

click_weight_tensor = torch.tensor(1.0, dtype=torch.float32, device=device)  # Ensure consistent type Click weight is always 1

In [35]:
# Map weights back to the original data
mapped_weights = booking_weight_tensor[mapped_srch_ids]

# Step 8: Assign final weights based on booking_bool
weight_tensor = torch.where(filtered_booking_bool == 1, mapped_weights, click_weight_tensor)

# Step 9: Compute base probability (1 / group_size)
group_size_tensor = total_interactions[mapped_srch_ids]  # Map total_interactions using indices
base_prob_tensor = 1.0 / group_size_tensor

# Step 10: Compute weighted probability
weighted_prob_tensor = base_prob_tensor * weight_tensor

# Step 11: Normalize probabilities within each srch_id
sum_weighted_prob = torch.zeros(len(unique_ids), dtype=torch.float32, device=device)
sum_weighted_prob.scatter_add_(0, mapped_srch_ids, weighted_prob_tensor)

# Compute final normalized probabilities
orig_prob_tensor = weighted_prob_tensor / sum_weighted_prob[mapped_srch_ids]

# Convert back to Pandas for compatibility
filtered_data = pd.DataFrame(filtered_tensor.cpu().numpy(), columns=data.drop(columns=["date_time"]).columns)
filtered_data["srch_id"] = filtered_srch_id.cpu().numpy()
filtered_data["orig_prob"] = orig_prob_tensor.cpu().numpy()

# Print the filtered DataFrame
print(filtered_data.head())


   srch_id  site_id  visitor_location_country_id  visitor_hist_starrating  \
0        1     12.0                        187.0                      NaN   
1        1     12.0                        187.0                      NaN   
2        1     12.0                        187.0                      NaN   
3        1     12.0                        187.0                      NaN   
4        1     12.0                        187.0                      NaN   

   visitor_hist_adr_usd  prop_country_id  prop_id  prop_starrating  \
0                   NaN            219.0    893.0              3.0   
1                   NaN            219.0  10404.0              4.0   
2                   NaN            219.0  21315.0              3.0   
3                   NaN            219.0  27348.0              2.0   
4                   NaN            219.0  29604.0              4.0   

   prop_review_score  prop_brand_bool  ...  comp7_rate  comp7_inv  \
0                3.5              1.0  ...     